<a href="https://colab.research.google.com/github/Bahae9/Micro_club_datathon2024/blob/main/Spam%20detection/MC_datathon_Spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading data

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d beatoa/spamassassin-public-corpus
# !unzip -q spamassassin-public-corpus.zip

 63% 7.00M/11.1M [00:00<00:00, 62.7MB/s]
100% 11.1M/11.1M [00:00<00:00, 62.6MB/s]


## Modules

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import email
import os
from bs4 import BeautifulSoup
import re
import xgboost as xgb

## Data preprocessing

In [ ]:
def get_clean_text(file):
  parsed_email = email.message_from_string(file.read())
  body = parsed_email.get_payload(decode=True)
  if body is not None:
    soup = BeautifulSoup(body, "html.parser")
    text = soup.get_text().lower()
    # Removing special characters, links, phone numbers and no needed elements like dashes and extra spaces
    text = re.sub(r'\n|\t|\s{2,}|[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}|https://\S+|http://\S+|www.\S+|-|\+\s*\d+(\s|\(\d+\)|\(\d+\)\s*|\s?\d+)*|\.{2,}|\*|_{3,}|!|\?|[^a-zA-Z0-9()\'\/\s]', '', text)
    text = text.replace('&', 'and').replace('$', 'dollars ')
    return text
  return None

In [ ]:
def add_text_to_dataframe(folder_path, status, dataframe):
  files = os.listdir(folder_path)
  for file_name in files:
      file_path = os.path.join(folder_path, file_name)
      if os.path.isfile(file_path):
          f = open(file_path, encoding="ISO-8859-1")
          dataframe.loc[len(dataframe.index)] = [status, (get_clean_text(f))]
  return dataframe

In [ ]:
data = pd.DataFrame(columns=['Category', 'Message'])
data

,Category,Message


### Adding cleaned data to dataframe

In [ ]:
data = add_text_to_dataframe('/content/easy_ham/easy_ham', 'ham', data)
data = add_text_to_dataframe('/content/hard_ham/hard_ham', 'ham', data)
data = add_text_to_dataframe('/content/spam_2/spam_2', 'spam', data)

<ipython-input-11-e7864167504f>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(body, "html.parser")
<ipython-input-11-e7864167504f>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(body, "html.parser")
<ipython-input-11-e7864167504f>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(body, "html.parser")
<ipython-input-11-e7864167504f>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(body, "html.parser")
<ipython-input-11-e7864167504f>:5: MarkupResemblesLocato

In [ ]:
data

,Category,Message
0,ham,mf here is a sample conversation on (msn messe...
1,ham,datewed 21 aug 2002 105446 0500fromchris garri...
2,ham,i need to experiment with ripping out all my s...
3,ham,url date not suppliedtonight on the style netw...
4,ham,normally i disdain any kind of mysticism even ...
...,...,...
4193,spam,what a loan fromlenders can do for yourates ha...
4194,spam,remove yourself from this recurring list bycli...
4195,spam,jokeadayhuman growth hormone precursor receive...
4196,spam,juice for your healthturn your favorite fruits...


### Cleaning and preprocessing dataframe

In [ ]:
def encode(df):
  df.dropna(inplace=True)
  df = df.sample(frac=1, random_state=42)
  df.replace({'ham': 0, 'spam': 1}, inplace=True)
  return df

In [ ]:
df = encode(data)

In [ ]:
X = df['Message']
y = df['Category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Text embedding

In [ ]:
feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)

X_train = feature_extraction.fit_transform(X_train)
X_test = feature_extraction.transform(X_test)

## Models and training

In [ ]:
RF_model = RandomForestClassifier()
RF_model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

SVC(probability=True)

In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('RF', RF_model), ('SVC', svm_model), ('xgb', xgb_model)],
    voting='soft',
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('RF', RandomForestClassifier()),
                             ('SVC', SVC(probability=True)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))],
                 voting='soft')

## Evaluation

In [ ]:
y_pred = voting_clf.predict(X_test)
acc = accuracy_score(y_pred, y_test)
acc

0.9883116883116884

### The best score got by using voting classifier, containing the models: Random forest, Support vector machine classifier and XGBoost classifier with an Accuracy score of: 0.9883116883116884